In [1]:
# from the website "https://www.nseindia.com/static/market-data/securities-available-for-trading" 
# Download - "Securities available for Equity segment (.csv) (147 KB)"

In [1]:
import pandas as pd, os, json
import yfinance as yf

In [2]:
stocks_df = pd.read_csv("../data/EQUITY_L.csv")
stocks_df.shape

(2212, 8)

In [3]:
stocks_df.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,EQ,03-MAY-1995,10,1,INE253B01015,10
2,360ONE,360 ONE WAM LIMITED,EQ,19-SEP-2019,1,1,INE466L01038,1
3,3IINFOLTD,3i Infotech Limited,EQ,22-OCT-2021,10,1,INE748C01038,10
4,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10


In [5]:
stocks_df["SYMBOL"].value_counts()

SYMBOL
20MICRONS     1
21STCENMGM    1
360ONE        1
3IINFOLTD     1
3MINDIA       1
             ..
ZOTA          1
ZUARI         1
ZUARIIND      1
ZYDUSLIFE     1
ZYDUSWELL     1
Name: count, Length: 2212, dtype: int64

In [6]:
for _, row in stocks_df.iterrows():
    # print(_)
    print(row["SYMBOL"], row["NAME OF COMPANY"])
    break

20MICRONS 20 Microns Limited


In [ ]:
mappings = { row["NAME OF COMPANY"]: row["SYMBOL"] for _, row in stocks_df.iterrows() }

os.makedirs("../assets", exist_ok=True)
json_output_path = "../assets/company_to_symbol.json"

with open(json_output_path, "w", encoding='utf-8') as f:
    if os.path.exists(json_output_path):
        print("You are about to rewrite this file. All the pre-existing content will be lost.")
        answer = input("Do you wish to continue? (y/n): ")
        answer = answer.lower()
        if answer == 'y':
            json.dump(mappings, f, indent=2, ensure_ascii=False)

print("Saved the json file successfully.")

Saved the json file successfully.


In [4]:
mappings = { row["SYMBOL"]: row["NAME OF COMPANY"] for _, row in stocks_df.iterrows() }

os.makedirs("../assets", exist_ok=True)
json_output_path = "../assets/symbol_to_company.json"

with open(json_output_path, "w", encoding='utf-8') as f:
    if os.path.exists(json_output_path):
        print("You are about to rewrite this file. All the pre-existing content will be lost.")
        answer = input("Do you wish to continue? (y/n): ")
        answer = answer.lower()
        if answer == 'y':
            json.dump(mappings, f, indent=2, ensure_ascii=False)

print("Saved the json file successfully.")

You are about to rewrite this file. All the pre-existing content will be lost.
Saved the json file successfully.


In [ ]:
sector_map = {}

for sym in stocks_df["SYMBOL"].tolist():
    try:
        info = yf.Ticker(f"{sym}.NS").info
        sector = info.get("sector", None)
        industry = info.get("industry", None)
        sector_map[sym] = {
            "sector": sector,
            "industry": industry
        }
        # print(sym, sector, industry)
    except:
        continue

with open("../assets/symbol_to_sector.json", "w") as f:
    if os.path.exists("../assets/symbol_to_sector.json"):
        print("You are about to rewrite this file. All the pre-existing content will be lost.")
        answer = input("Do you wish to continue? (y/n): ")
        answer = answer.lower()
        if answer == 'y':
            json.dump(sector_map, f, indent=2)


print("Saved the json file successfully.")

Saved the json file successfully.


In [ ]:
with open("../assets/symbol_to_sector.json", "r", encoding="utf-8") as f:
    symbol_to_sector = json.load(f) 

sector_to_symbols = {}

for sym, meta in symbol_to_sector.items():
    sec = (meta.get("sector") or "").strip()
    # print(sec)
    if not sec:
        continue
    sector_to_symbols.setdefault(sec.lower(), []).append(sym)

with open("../assets/sector_to_symbols.json", "w") as f:
    if os.path.exists("../assets/sector_to_symbols.json"):
        print("You are about to rewrite this file. All the pre-existing content will be lost.")
        answer = input("Do you wish to continue? (y/n): ")
        answer = answer.lower()
        if answer == 'y':
            json.dump(sector_to_symbols, f, indent=2)


print("Saved the json file successfully.")

Saved the json file successfully.


In [ ]:
REGULATOR_IMPACT_RULES = {
    "rbi": {"sectors": ["financial services","banking","nbfc"], "confidence": 0.6},
    "sebi": {"sectors": ["financial services","capital markets"], "confidence": 0.6},
    "trai": {"sectors": ["Telecom", "Communication Services", "Internet Services"], "confidence": 0.6},
    "irdai": {"sectors": ["Insurance", "Life Insurance", "Health Insurance"], "confidence": 0.6},
    "mnre": {"sectors": ["Renewable Energy", "Utilities", "Power"], "confidence": 0.5}
}

with open("../assets/regulator_impact_rules.json", "w") as f:
    if os.path.exists("../assets/regulator_impact_rules.json"):
        print("You are about to rewrite this file. All the pre-existing content will be lost.")
        answer = input("Do you wish to continue? (y/n): ")
        answer = answer.lower()
        if answer == 'y':
            json.dump(REGULATOR_IMPACT_RULES, f, indent=2)

print("Regulation mapping uploaded!")

Regulation mapping uploaded!


In [35]:
rule = REGULATOR_IMPACT_RULES.get("rbi")
rule

{'sectors': ['financial services', 'banking', 'nbfc'], 'confidence': 0.6}

In [43]:
with open("../assets/policy_impact_rules.json", "r") as f:
    POLICY_IMPACT_RULES = json.load(f)

In [45]:
POLICY_IMPACT_RULES["inflation"]

{'sectors': ['Consumer Staples',
  'Consumer Discretionary',
  'FMCG',
  'Retail',
  'Automobile'],
 'confidence': 0.5,
 'reason': 'Inflation affects consumer demand and input costs across sectors.'}